<a href="https://colab.research.google.com/github/Mattlee10/zone2/blob/main/25_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 목표

- HR/HRV 데이터를 활용해 Zone2 감지 + 회복 상태 분석
- 사용자 형태에 따른 간단한 텍스트 피드백 생성

2. 데이터

- Apple Health, Samsung Health CSV(심박수/HRV 포함)


3. 단계별 구현

- 데이터 전처리
  - CSV 로딩 -> 시간 정렬, 결측치 제거
  - HR -> 1분 단위 평균
  - HRV(SDNN or rMSSD) 계산 (연속 구간 필요 시 슬라이딩 윈도우 적용)

- Zone 2 감지
  - 최대 심박수 = 220 - 나이
  - Zone 2 범위 = 60-70% of max HR
  - 해당 구간에 있었던 시간 누적

- 회복/스트레스 분석: HRV 낮고 RHR 높으면 회복 저하
- 요약 텍스트 출력: Rule based or GPT 활용

4. 시각화

- matplotlib로 HR/HRV 변화 그래프
- Zone 2 운동 시간 히트맵
- 추후 Canva/Figma 시각화 연결 포인트 설정

5. 다음 단계

- 리포트 자동 생성(PDF 형태)
- 카카오 공유 메시지 생성 시나리오

# Zone 2 감지 알고리즘

데이터 입력: 웨어러블에서 수집한 시간별 심박수(HR) 데이터를 Pandas DataFrame으로 준비

최대심박수(MHR) 계산
MHR = 220 - 나이

Zone 2 구간 정의

- 최소값: zone2_min = MHR x 0.60
- 최대값: zone2_max = MHR x 0.70

Zone 2 감지: 각 심박수가 zone2_min과 zone2_max 사이에 있는지 불리언 플래그(in_zone2) 생성

구간 요약

- 지속 시간: in_zone2 == True인 행 수 (분 단위)
- 비율: 전체 행 대비 in_zone2 비중

In [1]:
import pandas as pd

# 1. 데이터 입력 (예시 데이터)
data = {
    'Time': pd.date_range(start='2025-05-12 07:00', periods=10, freq='T'),
    'HR':    [105, 110, 115, 125, 130, 140, 135, 120, 110, 100]
}
df = pd.DataFrame(data)

# 2. 최대심박수(MHR) 계산
age = 30  # 사용자 나이
mhr = 220 - age

# 3. Zone 2 구간 정의
zone2_min = mhr * 0.60
zone2_max = mhr * 0.70

# 4. Zone 2 감지
df['in_zone2'] = df['HR'].between(zone2_min, zone2_max)

# 5. 구간 요약
zone2_duration = df['in_zone2'].sum()  # Zone 2 지속 시간 (분)
zone2_ratio    = df['in_zone2'].mean() # Zone 2 비율

# 결과 출력
print(f"Max HR (MHR): {mhr} bpm")
print(f"Zone 2 범위: {zone2_min:.1f} ~ {zone2_max:.1f} bpm")
print(f"Zone 2 지속 시간: {zone2_duration}분")
print(f"Zone 2 비율: {zone2_ratio:.1%}")

Max HR (MHR): 190 bpm
Zone 2 범위: 114.0 ~ 133.0 bpm
Zone 2 지속 시간: 4분
Zone 2 비율: 40.0%


<ipython-input-1-f515ecd42f51>:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  'Time': pd.date_range(start='2025-05-12 07:00', periods=10, freq='T'),
